In [1]:
class EthereumFraudModeling:
    """
        Data: 
            - https://www.kaggle.com/datasets/vagifa/ethereum-frauddetection-dataset
        
        Notes:
            We can modularize our codes into functions in the en. This is just a 
            placeholder for now.
    """
    
    def __init__(self):
        self.import_data()
        self.models = []

    def import_data(self, data_name='ethereum'):
        if data_name == 'ethereum':
            self.data = pd.read_csv('./data/transaction_dataset.csv', index_col=0)
        

# 1. Libaries

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc, classification_report, plot_confusion_matrix, precision_recall_curve, f1_score

# 2. Data Reading

In [3]:
efm = EthereumFraudModeling()
print(efm.data.shape)

(9841, 50)


# 3. Data Exploration (by Saksham)

# 4. Data Cleaning (by Saksham)

# 5. Data Preparation (by Saksham)

# 6. Sampling (by Saksham)

## Method 1. SMOTE Sampling

## Method 2. 3-Way cross validation

# 7. Modeling

## Model 1. Supervised: Random Forest (by Akshay)

## Model 2. Supervised: Decision Tree (by Akshay)

## Model 3. Unsupervised: GMM (by Pratik)

## Model 4. Unsupervised: DBSCAN (by Pratik)

# 8. Model Evaluation (by Kenny)

### -------------------- Functions Delarations --------------------

In [4]:
# Helper: compute corresponding f1-score given thresholds
def compute_f1(y_test, y_pred, thresholds):
    scores = []
    for th in thresholds:
        th_y_pred = y_pred.copy()
        
        th_y_pred[th_y_pred >= th] = 0
        th_y_pred[th_y_pred < th] = 1
        scores.append(f1_score(y_test, th_y_pred))
    return scores


def plot_ROC(y_test, y_pred, plt_config, f1=False):
    fpr, tpr, threshold = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    
    scores = []
    if f1:
        scores = compute_f1(y_test, y_pred, thresholds)

    plt.figure(figsize=plt_config['figsize'])
    plt.title('ROC for ' + plt_config['model'])
    plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
    plt.plot([0,1], [0,1], 'y--')
    if f1:
        plt.plot(thresholds, scores, 'r')
    plt.ylabel('True Positive Rate / Score')
    plt.xlabel('False Positive Rate / Threshold')
    plt.legend(['ROC', 'Perfect Classifier', 'F1'])
    plt.show()


def plot_PRC(y_test, y_pred, plt_config, f1=False):
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)

    scores = []
    if f1:
        scores = compute_f1(y_test, y_pred, thresholds)

    plt.figure(figsize=plt_config['figsize'])
    plt.title('PRC for ' + plt_config['model'])
    plt.plot(recall, precision, 'b')
    if f1:
        plt.plot(thresholds, scores, 'r')
    plt.ylabel('Precision / Score')
    plt.xlabel('Recall / Threshold')
    plt.legend(['PRC', 'F1'])
    plt.show()


## Metric 1. ROC w/ F1

In [5]:
# Prepare parameters for ROC

# Uncomment the following when you have data ready

"""
# Get probablistic predictions using a trained model
probs = xgb_c.predict_proba(norm_test_f)
pred = probs[:, 1]

# Customize plot
plt_config = {
    'figsize': (12, 8),
    'model': 'XGB Classifier'
}

# Call the function with your choice of f1 plotting (in the same grapgh)
plot_ROC(y_test, pred, plt_config, f1=True)
"""

"\n# Get probablistic predictions using a trained model\nprobs = xgb_c.predict_proba(norm_test_f)\npred = probs[:, 1]\n\n# Customize plot\nplt_config = {\n    'figsize': (12, 8),\n    'model': 'XGB Classifier'\n}\n\n# Call the function with your choice of f1 plotting (in the same grapgh)\nplot_ROC(y_test, pred, plt_config, f1=True)\n"

## Metric 2. PRC w/ F1

In [6]:
# Prepare parameters for PRC

# Uncomment the following when you have data ready

"""
# Get probablistic predictions using a trained model
probs = xgb_c.predict_proba(norm_test_f)
pred = probs[:, 1]

# Customize plot
plt_config = {
    'figsize': (12, 8),
    'model': 'XGB Classifier'
}

# Call the function with your choice of f1 plotting (in the same grapgh)
plot_PRC(y_test, pred, plt_config, f1=True)
"""

"\n# Get probablistic predictions using a trained model\nprobs = xgb_c.predict_proba(norm_test_f)\npred = probs[:, 1]\n\n# Customize plot\nplt_config = {\n    'figsize': (12, 8),\n    'model': 'XGB Classifier'\n}\n\n# Call the function with your choice of f1 plotting (in the same grapgh)\nplot_PRC(y_test, pred, plt_config, f1=True)\n"